In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression, Ridge
from sklearn import datasets as ds
from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def gen_helix_data(n, dim):
    t = 10 - np.random.rand(n) * 20
    helix_data = pd.DataFrame([np.cos(t), np.sin(t), t]).T
    return helix_data

def gen_sphere_data(n, dim):
    norm_x = np.random.randn(n,dim)
    sphere_data = pd.DataFrame(norm_x/np.sqrt(np.sum(norm_x**2, axis=1))[:,None])
    return sphere_data

def gen_roll_data(n, dim):
    roll_data = pd.DataFrame(ds.make_swiss_roll(n)[0])
    return roll_data

def gen_norm_data(n, dim):
    norm_xyz = np.random.multivariate_normal(np.zeros(dim), np.identity(dim), n)
    norm_data = pd.DataFrame(norm_xyz)
    return norm_data

def gen_uniform_data(n, dim):
    uniform_xyz = np.random.uniform(size=(n, dim))
    uniform_data = pd.DataFrame(uniform_xyz)
    return uniform_data

In [6]:
def m_k_hat(x_center, k, data = None, knn_model = None):
    if knn_model is None:
        knn_model = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(data)
    dist, ind = knn_model.kneighbors(x_center.reshape(1,-1))
    d = np.log(dist[0,-1]/dist[0,:-1]).sum()/(k-2)
    return 1./d,dist[0,-1]

def bootstrap_m_hat(X, nb_iter=100, random_state=None, k1 = 3, k2 = 10):

    if random_state is None:
        rng = np.random
    else:
        rng = np.random.RandomState(random_state)
    
    nb_examples = X.shape[0]
    
    
    Rs = []
    dims = []

    for i in range(nb_iter):
        
        ind_center = rng.randint(0, nb_examples - 1, size=1)
        x_center = X.values[ind_center]
        idx = np.unique(rng.randint(0, nb_examples - 1, size=nb_examples))
        X_bootstrap = X.iloc[idx[idx != ind_center]]

        for k in range(k1,k2+1):    
            dim, R = m_k_hat(x_center, k, X_bootstrap)
            dims.append(dim)
            Rs.append(R)
                
    return np.array(Rs), np.array(dims)

def reg_reg_reg(data):
    
    def _fit_poly_reg(fit_dim, fit_R, alpha=3):
        fit_dim = np.array(fit_dim)
        fit_R = np.array(fit_R)

        fit_R_alpha = fit_R.reshape(-1, 1) ** alpha
        lm = LinearRegression()
        lm.fit(fit_R_alpha, fit_dim)

        return lm.intercept_, lm.coef_[0]

    
    finalfinal = []

    for i in range(10):
        final = []
        for i in range(10):

            result = []
            for i in range(10):
                Rs, dims = bootstrap_m_hat(data, nb_iter=100)
                fit_Rs = np.linspace(Rs.min(), Rs.max(), 10)
                fit_dims = np.array([dims[(Rs >= r1) & (Rs <= r2)].mean() for r1, r2 \
                         in zip(fit_Rs, fit_Rs[1:].tolist() + [Rs.max() + 1])])
                fit_dims[np.isnan(fit_dims) | np.isinf(fit_Rs)] = data.shape[1]
                result.append([_fit_poly_reg(fit_dims, fit_Rs)[0], np.mean(fit_dims)])

            final.append(np.mean(result, axis=0))
        finalfinal.append(np.mean(final, axis=0))
    return finalfinal

In [7]:
REG_means = []
LB_means = []
REG_stds = []
LB_stds = []
for dim in tqdm_notebook(range(1, 101)):
    data = gen_sphere_data(1000, dim)
    result = reg_reg_reg(data)
    
    mean = np.mean(result, axis=0)
    std = np.std(result, axis=0)
    
    REG_means.append(mean[0])
    LB_means.append(mean[1])
    REG_stds.append(std[0])
    LB_stds.append(std[1])

A Jupyter Widget

/Users/marinagomtsyan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
/Users/marinagomtsyan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Mean of empty slice.
/Users/marinagomtsyan/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
